In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pyfolio as pf
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

/home/aviral/Desktop/Algo_trading_iitk/env/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/aviral/Desktop/Algo_trading_iitk/env/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
class TestStrategy(bt.Strategy):
    
    params = dict(profit_percent=0.3, loss_percent = 0.2, efast = 10, eslow=30)
    
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        tm = self.datas[0].datetime.time(0)
        print('%s, %s,  %s' % (dt.isoformat(), tm.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.time = self.datas[0].datetime.time
        self.order = None
        self.price = None
        self.comm = None
        emafast = bt.ind.EMA(period=self.p.efast)
        emaslow = bt.ind.EMA(period=self.p.eslow)
        self.crossover = bt.ind.CrossOver(emafast,emaslow)
        
    def notify_order(self, order):
        date = self.data.datetime.datetime().date()
        
        if order.status in [order.Submitted, order.Accepted]:
            return
            
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.price = order.executed.price
                self.comm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Margin, order.Rejected]:
            self.log('Order Margin/Rejected')
        
        elif order.status in [order.Canceled]:
            self.log('Order Canceled')
            #Closing Bracket orders comes under this condition

        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
                 
    def next(self):
        #self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return
        
        #adding the 15 minute buffer
        if (self.data.datetime.time() < datetime.time(9,30)) or (self.data.datetime.time() > datetime.time(15,15)):
            return
        
        if not self.position:
            #Check the order while placing the amounts for different types, limit order and to do functions
            #like find a breakout and confirm the indicator. Many a times generates errors
            if self.crossover>0:
                #Crossover is an object and not a numerical value, no point in printing it
                self.log('Buying due to crossover')
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.buy_bracket(limitprice=entry*(1+(self.p.profit_percent/100)),
                                              stopprice=entry*(1-(self.p.loss_percent/100)),
                                              exectype=bt.Order.Market)

        #Uncomment this and find that it is buying and selling way too frequently.
        #The behavior can be observed from the plot but no explanation
        else:
            if self.crossover<0:
                self.log('Selling due to crossover')
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                entry = self.dataclose
                self.order = self.sell_bracket(limitprice=entry*(1-(self.p.profit_percent/100)),
                                              stopprice=entry*(1+(self.p.loss_percent/100)),
                                              exectype=bt.Order.Market)

        

In [3]:
#A function for properly processing and printing analyzers
def printTradeAnalysis(analyzer):
    
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("\n\nTrade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [4]:
if __name__ == '__main__':
    orig_stdout = sys.stdout
    f = open('/home/aviral/Desktop/Algo_trading_iitk/intraday_output.txt', 'w')
    sys.stdout = f
    
    cerebro = bt.Cerebro()
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, 'datas/orcl-1995-2014.txt')
    dirpath = os.path.dirname('/home/aviral/Desktop/Algo_trading_iitk/')
    datapath = os.path.join(dirpath, 'Nifty_2013.csv')

    #Check the format and positions of the datafeeds in case another file is used
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        
        #Even though the data file has 2013 in name, the data is actually of 2015
        fromdate=datetime.datetime(2015, 1, 1),
        todate=datetime.datetime(2015, 12, 31),
        nullvalue = 0.0,
        
        dtformat = ('%d/%m/%Y'),
        tmformat = ('%H:%M:%S'),
        
        timeframe = bt.TimeFrame.Minutes,
        compression = 1,
        #Compression is useful if the data is per minute but we want to use 5 min intervals
                
        datetime = 0,
        time = 1,
        open = 2,
        high = 3,
        low = 4,
        close = 5,
        volume = 6,
        openinterest = -1,
        reverse=False)

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)

    cerebro.addsizer(bt.sizers.FixedSize, stake=2)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    strats = cerebro.run()
    printTradeAnalysis(strats[0].analyzers.ta.get_analysis())

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    #A weird error, scroll down and check. After september 8th all orders are displaying margin/cancelled
    sys.stdout = orig_stdout
    f.close()


In [5]:
cerebro.plot()
#Observe that a lot more green and red markers for buying and selling respectively
#Even at the points where there is no crossover whatsoever, and looks like the buy and sell signs are executed also

<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 5 Axes>]]

In [6]:
#The analysing ratios and graphs
#Get the first strategy, in backtrader the analysing is done for each individual sttrategy
strat = strats[0]

#Pyfolio is the default name given by backtrader, check how to give a custom name
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

pf.create_full_tear_sheet(
    returns,
    live_start_date=datetime.datetime(2015,10,1))


IndexError: index -1 is out of bounds for axis 0 with size 0